In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
df_train = pd.read_csv('/content/drive/MyDrive/nn_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/nn_test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df_train_x = df_train.iloc[:, 1:-2]
y_binary = df_train.iloc[:,-2]
df_test_x = df_test.iloc[:, 1:]
X = (df_train_x - df_train_x.mean())/df_train_x.std()
x_test = (df_test_x - df_train_x.mean())/df_train_x.std()
print(X.shape)
print(x_test.shape)

(80000, 1024)
(20000, 1024)


In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [7]:
def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [8]:
def initialize_parameters(input_size, hidden_size, output_size):
    W1 = np.random.randn(input_size + 1, hidden_size) * 0.01
    W2 = np.random.randn(hidden_size + 1, output_size) * 0.01
    return W1, W2

In [9]:
def forward_propagation(X, W1, W2):
    X1 = np.hstack((X, np.ones((X.shape[0], 1))))
    Z1 = np.dot(X1, W1)
    A1 = sigmoid(Z1)
    A1_bias = np.hstack((A1, np.ones((A1.shape[0], 1))))
    Z2 = np.dot(A1_bias, W2)
    A2 = sigmoid(Z2)
    bla = {"X1": X1, "Z1": Z1, "A1": A1_bias, "Z2": Z2, "A2": A2}
    return A2, bla

In [10]:
def backward_propagation(y, bla, W2):
    m = y.shape[0]
    A1, A2 = bla["A1"], bla["A2"]
    dZ2 = A2 - y.reshape(-1, 1)
    dW2 = np.dot(A1.T, dZ2) / m
    dZ1 = np.dot(dZ2, W2[:-1].T) * sigmoid_derivative(bla["Z1"])
    dW1 = np.dot(bla["X1"].T, dZ1) / m
    grads = {"dW1": dW1, "dW2": dW2}
    return grads

In [11]:
def update_parameters(W1, W2, grads, learning_rate):
    W1 -= learning_rate * grads["dW1"]
    W2 -= learning_rate * grads["dW2"]
    return W1, W2

In [12]:
def train(X, y, hidden_size, learning_rate, iterations):
    input_size = X.shape[1]
    output_size = 1
    W1, W2 = initialize_parameters(input_size, hidden_size, output_size)
    losses = []
    for iteration in range(iterations):

        A2, bla = forward_propagation(X, W1, W2)
        loss = -np.mean(y * np.log(A2) + (1 - y) * np.log(1 - A2))
        losses.append(loss)

        grads = backward_propagation(y, bla, W2)

        W1, W2 = update_parameters(W1, W2, grads, learning_rate)

        if iteration % 100 == 0:
            print(f"Iteration {iteration}: Loss = {loss:.4f}")

    return W1, W2, loss

In [13]:
def predict(X, W1, W2):
    A2, _ = forward_propagation(X, W1, W2)
    return (A2 > 0.5).astype(int)

In [ ]:
if __name__ == "__main__":
  hidden_size = 4
  learning_rate = 0.1
  iterations = 1000
  W1, W2, losses = train(X, y_binary, hidden_size, learning_rate, iterations)


In [ ]:
plt.plot(losses)
    plt.title("Loss Curve")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()